In [1]:
import os
import re
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as im, ImageDraw, ImageFont
import PIL
from glob import glob
from utils import add_margin, expand2square, put_in_a_box, binarize
from tqdm import tqdm

from typing import *

In [2]:
PATH_TO_FONT_CHARS = "../formatted_data/chars_images/chars"
PATH_TO_AUGMENTED_CHARS = "../formatted_data/dataset_more_data"

ttfs = glob("ttfs/*.ttf")

LOWERCASE = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
UPPERCASE = re.sub("[ЪЫЙЬ]", '', LOWERCASE.upper())

names = {
#     ',': "coma",
#     ":": "colon",
#     ".": "dot",
    "-": "dash",
    "?": "q_mark",
    "!": "e_mark",
#     "…": "tridot",
}
PUNCT_POOL=list(names.keys())
PUNCT_DIRS=list(names.values())

In [10]:
for l in list(LOWERCASE) + PUNCT_DIRS + ["trash"]:
    os.makedirs(f"{PATH_TO_AUGMENTED_CHARS}/{l}", exist_ok=1)    

In [4]:
# class Char:
#     def __init__(self, path: str):
#         self.path = path
#         self.image = im.open(path)
        
#         split = os.path.normpath(path).split(os.path.sep)
#         self.name = split[-1].replace(".png", "")
#         self.font = split[-2]


In [5]:
# fonts = os.listdir(PATH_TO_FONT_CHARS)
# chars_dict = {font: glob(f"{PATH_TO_FONT_CHARS}/{font}/*") for font in fonts}
# for k, v in chars_dict.items():
#     chars = []
#     for char in v:
#         chars.append(Char(char))
#     chars_dict[k] = chars

In [7]:
def convert_png_transparent(image, bg_color=(255,255,255)):
    array = np.array(image, dtype=np.ubyte)
    mask = (array[:,:,:3] == bg_color).all(axis=2)
    alpha = np.where(mask, 0, 255)
    array[:,:,-1] = alpha
    return im.fromarray(np.ubyte(array))

def get_dominant_color(img):
    palette_size = 2
    # Resize image to speed up processing
    #img = convert_png_transparent(pil_img.copy())
    img.thumbnail((100, 100))

    # Reduce colors (uses k-means internally)
    paletted = img.convert('P', palette=im.ADAPTIVE, colors=palette_size)

    # Find the color that occurs most often
    palette = paletted.getpalette()
    color_counts = sorted(paletted.getcolors(), reverse=False)
    palette_index = color_counts[0][1]
    dominant_color = palette[palette_index*3:palette_index*3+3]
    dominant_color.append(255)

    return tuple(dominant_color)

def expand(img, top=0, right=0, bottom=0, left=0):
    return add_margin(
             expand2square(img),
             top=top,
             bottom=bottom,
             left=left,
             right=right
           )

def rotate(degree, image):
    return image.rotate(degree, expand=True, fillcolor=(255,255,255), resample=PIL.Image.Resampling.BICUBIC)

def closest_to(char_image, coord: Tuple[int, int]):
    bin_img = binarize(char_image)
    h, w = char_image.size
    distances = list()
    for r_idx in range(w):
        for c_idx in range(h):
            if bin_img[r_idx, c_idx] > 0.5:
                distance = euclideanDistance(coord, (r_idx, c_idx))
                distances.append((r_idx, c_idx, distance))
    return min(distances, key=lambda entry: entry[2])[:2]

def resize(image: "Image", what: str, how_much: float):
    w,h = image.width, image.height
    if what == 'width':  return image.resize((round(w * how_much), h))
    if what == 'height': return image.resize((w, round(h * how_much)))
    raise AssertionError(f"`what` must be either \"width\" or \"height\", not {what}")
    
def euclideanDistance(coordinate1, coordinate2):
    return pow(pow(coordinate1[0] - coordinate2[0], 2) + pow(coordinate1[1] - coordinate2[1], 2), .5)

def fill_background_png(image):
    fill_color = (255,255,255)  # your new background color
    if image.mode in ('RGBA', 'LA'):
        background = im.new(image.mode[:-1], image.size, fill_color)
        background.paste(image, image.split()[-1]) # omit transparency
        image = background
    return image

def write(font, word):
    font = ImageFont.truetype(font,64)
    img  = im.new("RGBA", (500,150),(255,255,255))
    ImageDraw.Draw(img).text((22, 22), word,(49, 76, 175), font=font)
    return put_in_a_box(img)

In [166]:
""" MOVING UP AND DOWN """
def move_up_and_down(save_path, pool):
    for font in ttfs:
        fontname = os.path.basename(font).replace(".ttf", '')
        for letter in pool:
            try:
                char = write(font, letter)
            except:
                print(letter)
                return
            for margin, side in [(m, side) for m in range(5,16,3) for side in "rltb"]:
                if side == "l":
                    image = expand(char, left=margin)
                elif side == "r":
                    image = expand(char, right=margin)
                elif side == "t":
                    image = expand(char, top=margin)
                elif side == "b":
                    image = expand(char, bottom=margin)
                image.convert("RGB").save(f"{save_path}/{letter.lower()}/{letter}_{fontname}_{side}{margin}.jpg")

move_up_and_down(PATH_TO_AUGMENTED_CHARS, LOWERCASE+UPPERCASE)

In [167]:
""" ROTATION FROM -40° to 40° """
def rotation_augment(save_path, pool):
    for font in ttfs:
        fontname = os.path.basename(font).replace(".ttf", '')
        for letter in pool:
            char = write(font, letter)
            for degree in range(-40, 21, 5):
                image = rotate(degree, char)
                image.convert("RGB").save(f"{save_path}/{letter.lower()}/{fontname}_{letter}_{degree}.jpg")
rotation_augment(PATH_TO_AUGMENTED_CHARS, LOWERCASE+UPPERCASE)

In [168]:
# goes up and right from here
right_bottom_spot = lambda image: (closest_to(image, (image.height, image.width)), {"x": 100, "y": -30})
# goes down and right from here
right_top_spot = lambda image: (closest_to(image, (0, image.height)), {"x": 100, "y": 30})

# goes up and rigth from here
left_bottom_spot = lambda image: (closest_to(image, (image.width, 0)), {"x": 100, "y": -30})
# goes down and left from here
left_top_spot = lambda image: (closest_to(image, (0,0)), {"x": -100, "y": 30})

In [ ]:
# """ CONNECTED CHARS """
# def connected_chars(save_path, pool):
#     for font in ttfs:
#         fontname = os.path.basename(font).replace(".ttf", '')
#         for letter in pool:
#             char = write(font, letter)
#             dominant = get_dominant_color(char)
            
#             # left_top from prev
#             char_copy = char.copy()
#             draw = ImageDraw.Draw(char_copy) 
#             (y, x), destination = left_top_spot(char_copy)
#             draw.line((x,y, x+destination["x"],y+destination["y"]), fill=dominant, width=3, joint="curve")     
#             char_copy.convert("RGB").save(f"{save_path}/{letter.lower()}/{fontname}_{letter}_lt.jpg")

#             # right_top to next
#             char_copy = char.copy()
#             draw = ImageDraw.Draw(char_copy)
#             (y, x), destination = right_top_spot(char_copy)
#             draw.line((x,y, x+destination["x"],y+destination["y"]), fill=dominant, width=3, joint="curve")
#             char_copy.convert("RGB").save(f"{save_path}/{letter.lower()}/{fontname}_{letter}_rt.jpg")

#             # right_bottom to next
#             char_copy = char.copy()
#             draw = ImageDraw.Draw(char_copy) 
#             (y, x), destination = right_bottom_spot(char_copy)
#             draw.line((x,y, x+destination["x"],y+destination["y"]), fill=dominant, width=3, joint="curve")
#             char_copy.convert("RGB").save(f"{save_path}/{letter.lower()}/{fontname}_{letter}_rb.jpg")

#             # left_bottom to next
#             char_copy = char.copy()
#             draw = ImageDraw.Draw(char_copy) 
#             (y, x), destination = left_bottom_spot(char_copy)
#             draw.line((x,y, x+destination["x"],y+destination["y"]), fill=dominant, width=3, joint="curve")
#             char_copy.convert("RGB").save(f"{save_path}/{letter.lower()}/{fontname}_{letter}_lb.jpg")

# connected_chars(PATH_TO_AUGMENTED_CHARS, LOWERCASE+UPPERCASE)

In [169]:
""" RESIZED, i.e. squashed or extended"""
def resized_augment(save_path, pool):
    for font in ttfs:
        fontname = os.path.basename(font).replace(".ttf", '')
        for letter in pool:
            char = write(font, letter)
            w, h = char.size
            for i in np.arange(0, 0.81, 0.1):
                for dim in ["width", "height"]:
                    perc = int(i * 100)
                    resize(char, dim, 1+i).convert("RGB")\
                        .save(f"{save_path}/{letter.lower()}/{letter}_{fontname}_{perc}%more_{dim}.jpg")
            for i in np.arange(0, 0.61, 0.1):
                for dim in ["width", "height"]:
                    perc = int(i * 100)
                    resize(char, dim, 1-i).convert("RGB")\
                        .save(f"{save_path}/{letter.lower()}/{letter}_{fontname}_{perc}%less_{dim}.jpg")
            
resized_augment(PATH_TO_AUGMENTED_CHARS, LOWERCASE)

Создаём фейковые изображения символов в связке с соседними символами и называем это умным словом "аугментация"

In [8]:
def width_of(font, word):
    word = re.sub("[вбзудфзр]", "о", word)
    return write(font, word).width

In [174]:
LOWERCASE

'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

In [173]:
""" SIMULATING A CONTEXT FOR A CHAR """
import time
def simulate(save_path,
             target_char_pool=LOWERCASE,
             left_neighbor_pool=LOWERCASE,
             right_neighbor_pool=LOWERCASE,
             simulations_per_char=33,
             description='simulation'):
    for font in ttfs:
        fontname = os.path.basename(font).replace(".ttf", '')
        print(fontname)
        for s in tqdm(target_char_pool):
            for f in left_neighbor_pool:
                print(f, end=" ")
                randint = np.random.randint(-50, 50)
                for t in right_neighbor_pool:
                    try:
                        word = f"{f}{s}{t}"
                        img = write(font, word)
                        random = int(time.time() * 1000) % (randint if randint != 0 else 50)

                        r = img.width - width_of(font, f"{t}")
                        l = width_of(font, f"{f}")
                        s_itself = width_of(font, s)

                        # widen the borders randomly
                        margin = 5 + int(time.time() * 1000) % 5
                        l, r = l-margin, r+margin
                        img = img.crop((l, 0, r, img.height))

                        ## Applying mutations to widen the variety of images
                        # 40% chance to rotate
                        if 15 <= abs(random) <= 35:
                            img = rotate(random // 2, img)
                        # 50% chance to expand or shrink an image in either of two dimensions
                        if random % 2 == 0: 
                            img = resize(image=img,
                                         what="width" if abs(random) <= 25 else "height",
                                         how_much=(120 + random) / 100)
                        fill_background_png(img).convert('RGB').save(f"{save_path}/{s.lower()}/{f+s+t}_{fontname}_{word}.jpg")
                    except Exception as e:
                        print(f"\n{e}\n")
                
simulate(PATH_TO_AUGMENTED_CHARS)

Abram


  0%|                                                                                                                                                                                                                                                    | 0/33 [00:00<?, ?it/s]

а б в г д е ё ж з и й к л м н о п р с т у ф х ц ч ш щ ъ ы ь э ю я 

  3%|███████▏                                                                                                                                                                                                                                    | 1/33 [00:29<15:35, 29.24s/it]

а б в г д е ё ж з и й к л м н о п р с т у ф х ц ч ш щ ъ ы ь э ю я 

...

 94%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊              | 31/33 [14:18<00:55, 27.69s/it]

а б в г д е ё ж з и й к л м н о п р с т у ф х ц ч ш щ ъ ы ь э ю я 

 97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉       | 32/33 [14:46<00:27, 27.68s/it]

а б в г д е ё ж з и й к л м н о п р с т у ф х ц ч ш щ ъ ы ь э ю я 

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [15:13<00:00, 27.70s/it]


In [178]:
""" PUNCTUTATION MARKS """

left_neighbor_pool= re.sub("[вбзудфзр]", "", LOWERCASE)
def punctutaion_aug(save_path, pool):
    for font in ttfs:
        fontname = os.path.basename(font).replace(".ttf", '')
        print(fontname)
        for s in tqdm(pool):
            subdir = names[s]
            os.makedirs(f"{save_path}/{subdir}", exist_ok=1)
            for idx in range(70,270):
                try:
                    random = np.random.randint(-50, 50)

                    f = left_neighbor_pool[random % len(left_neighbor_pool)]
                    word = f"{f}{s} "

                    img = write(font, word)

                    l = img.width - width_of(font, f"{s} ") - 3
                    r = width_of(font, f"{f+s}") + 3
                    img = img.crop((l, 0, r, img.height))

                    ## Applying mutations to widen the variety of images
                    # 40% chance to rotate
                    if 15 <= abs(random) <= 35:
                        img = rotate(random // 2, img)
                    # 50% chance to expand or shrink an image in either of two dimensions
                    if random % 2 == 0: 
                        img = resize(image=img,
                                     what="width" if abs(random) >= 25 else "height",
                                     how_much=(140 + random*0.8) / 100)
                    fill_background_png(img).convert('RGB').save(f"{save_path}/{subdir}/{s}_{fontname}_{idx}.jpg")
                except Exception as e:
                    pass

punctutaion_aug(PATH_TO_AUGMENTED_CHARS, PUNCT_POOL)

Abram


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:11<00:00,  4.00s/it]


Propisi


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:13<00:00,  4.66s/it]


Gogol


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:13<00:00,  4.61s/it]


Denistina


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:13<00:00,  4.36s/it]


Tibalt


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:12<00:00,  4.02s/it]


In [13]:
""" TRASH  """
def trash(save_path, pool):
    for font in ttfs:
        randint = np.random.randint(150)
        fontname = os.path.basename(font).replace(".ttf", '')
        print(fontname)
        for idx in tqdm(range(len(pool))):
            for idx2 in range(len(pool)):
                for span in np.arange(0.4, 0.51, 0.05):
                    one = LOWERCASE[idx]
                    two = LOWERCASE[idx2]
                    word = f"{one + two}"
                    img = write(font, word)
                    l, r = img.width * span, img.width * (span+0.2)
                    img = fill_background_png(img).convert('RGB').crop((l, 0, r, img.height))
                    img.save(f"{save_path}/trash/{one}{two}_{fontname}_{round(span, 2)}.jpg")
trash(PATH_TO_AUGMENTED_CHARS, LOWERCASE)

Abram


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:22<00:00,  1.44it/s]


Propisi


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:25<00:00,  1.31it/s]


Gogol


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:24<00:00,  1.35it/s]


Denistina


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:25<00:00,  1.29it/s]


Tibalt


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:26<00:00,  1.25it/s]
